In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover,StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import RandomForestClassifier

In [2]:
spark= SparkSession.builder.config("spark.storage.memoryFraction", "0.6").appName('Twitter_rf').getOrCreate()
training_path = 'twitter_training.csv'
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("game", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("tweet", StringType(), True)
])

24/05/01 16:00:08 WARN Utils: Your hostname, sasamg-HP-Laptop-15s-eq2xxx resolves to a loopback address: 127.0.1.1; using 192.168.100.236 instead (on interface wlo1)
24/05/01 16:00:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/01 16:00:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataset = spark.read.csv(training_path, inferSchema=True, schema = schema)
dataset = dataset.dropna(subset=["tweet"])

In [4]:
train_set, test_set = dataset.randomSplit([0.8, 0.2], seed=42)

In [5]:
tokenizer = Tokenizer(inputCol="tweet", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="TF")
idf = IDF(inputCol="TF", outputCol="features")
label = StringIndexer(inputCol="sentiment", outputCol="label", handleInvalid="skip")

In [20]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features",maxDepth=15, numTrees=100)
pipeline_rf = Pipeline(stages=[tokenizer, hashingTF, idf, label, rf])

# Fit the model to the training data
rf_model = pipeline_rf.fit(train_set)

# Make predictions on the test data
rf_predictions = rf_model.transform(test_set)

24/05/01 16:39:25 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/05/01 16:39:25 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/05/01 16:39:27 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB
24/05/01 16:40:14 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB
24/05/01 16:40:14 WARN DAGScheduler: Broadcasting large task binary with size 7.9 MiB
24/05/01 16:40:15 WARN MemoryStore: Not enough space to cache rdd_209_2 in memory! (computed 113.1 MiB so far)
24/05/01 16:40:15 WARN BlockManager: Persisting block rdd_209_2 to disk instead.
24/05/01 16:40:15 WARN MemoryStore: Not enough space to cache rdd_209_1 in memory! (computed 113.1 MiB so far)
24/05/01 16:40:15 WARN BlockManager: Persisting block rdd_209_1 to disk instead.
24/05/01 16:40:15 WARN MemoryStore: Not enough space to cache rdd_209_0 in memory! (computed 113.1 MiB so far)
24/05/01 16:40:15 WARN BlockManager: Persisting block rdd_209_0 to disk 

In [21]:
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
evaluator_weighted_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_weighted_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

In [22]:
accuracy_rf = evaluator_accuracy.evaluate(rf_predictions)
f1_score_rf = evaluator_f1.evaluate(rf_predictions)
weighted_precision_rf = evaluator_weighted_precision.evaluate(rf_predictions)
weighted_recall_rf = evaluator_weighted_recall.evaluate(rf_predictions)

24/05/01 16:57:30 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
24/05/01 16:57:31 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
24/05/01 16:57:32 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
24/05/01 16:57:32 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


In [23]:
print("\n ********************************************** \n")
print("Accuracy:", accuracy_rf)
print("F1 Score:", f1_score_rf)
print("Weighted Precision:", weighted_precision_rf)
print("Weighted Recall:", weighted_recall_rf)
print("\n ********************************************** \n")


 ********************************************** 

Accuracy: 0.3947080541501436
F1 Score: 0.3057317510172687
Weighted Precision: 0.6702706290107006
Weighted Recall: 0.39470805415014354

 ********************************************** 

